## 1. 생성자

In [1]:
import numpy as np
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LeakyReLU, Reshape, Conv2D, Conv2DTranspose, Flatten, Dropout
from tensorflow.keras.optimizers import RMSprop

In [2]:
latent_dim = 32
height = 32
width = 32
channels = 3
generator_input = Input(shape=(latent_dim,))
# 입력을 16 × 16 크기의 128개 채널을 가진 특성 맵으로 변환
x = Dense(128 * 16 * 16)(generator_input)
x = LeakyReLU()(x)
x = Reshape((16, 16, 128))(x)
# 합성곱 층을 추가
x = Conv2D(256, 5, padding='same')(x)
x = LeakyReLU()(x)
# 32 × 32 크기로 업샘플링
x = Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = LeakyReLU()(x)
# 합성곱 층을 더 추가
x = Conv2D(256, 5, padding='same')(x)
x = LeakyReLU()(x)
x = Conv2D(256, 5, padding='same')(x)
x = LeakyReLU()(x)
# 32 × 32 크기의 3개 채널을 가진 특성 맵을 생성
x = Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = Model(generator_input, x)
generator.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
dense (Dense)                (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32768)             0         
_________________________________________________________________
reshape (Reshape)            (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 32, 32, 256)       104883

## 2. 판별자

In [3]:
discriminator_input = Input(shape=(height, width, channels))
x = Conv2D(128, 3)(discriminator_input)
x = LeakyReLU()(x)
x = Conv2D(128, 4, strides=2)(x)
x = LeakyReLU()(x)
x = Conv2D(128, 4, strides=2)(x)
x = LeakyReLU()(x)
x = Conv2D(128, 4, strides=2)(x)
x = LeakyReLU()(x)
x = Flatten()(x)

# 드롭아웃 층을 넣는 것이 아주 중요합니다!
x = Dropout(0.4)(x)

# 분류 층
x = Dense(1, activation='sigmoid')(x)

discriminator = Model(discriminator_input, x)
discriminator.summary()

discriminator_optimizer = RMSprop(lr=0.0008, clipvalue=1.0, decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 6, 6, 128)         0   

## 3. 적대적네트워크

In [4]:
# 판별자의 가중치가 훈련되지 않도록 설정
discriminator.trainable = False

gan_input = Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = Model(gan_input, gan_output)

gan_optimizer = RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')
gan.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
model (Functional)           (None, 32, 32, 3)         6264579   
_________________________________________________________________
model_1 (Functional)         (None, 1)                 790913    
Total params: 7,055,492
Trainable params: 6,264,579
Non-trainable params: 790,913
_________________________________________________________________


## 4. Training

In [5]:
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.datasets import cifar10

# CIFAR10 데이터를 로드
(x_train, y_train), (_, _) = cifar10.load_data()

# 자동차 이미지를 선택(클래스 1)
x_train = x_train[y_train.flatten() == 1]

# 데이터를 정규화
x_train = x_train.reshape(
    (x_train.shape[0],) + (height, width, channels)).astype('float32') / 255.

iterations = 10000
batch_size = 20

try:
    os.mkdir('gan_cars')
except:
    print('')

save_dir = 'gan_cars/'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

# 훈련 반복 시작
start = 0
for step in range(iterations):
    # 잠재 공간에서 무작위로 포인트를 샘플링
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # 가짜 이미지를 디코딩
    generated_images = generator.predict(random_latent_vectors)

    # 진짜 이미지와 연결
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])

    # 진짜와 가짜 이미지를 구분하여 레이블을 병합
    labels = np.concatenate([np.ones((batch_size, 1)),
                             np.zeros((batch_size, 1))])
    
    # 레이블에 랜덤 노이즈를 추가합니다. 아주 중요!
    labels += 0.05 * np.random.random(labels.shape)

    # discriminator를 훈련
    d_loss = discriminator.train_on_batch(combined_images, labels)

    # 잠재 공간에서 무작위로 포인트를 샘플링
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # 모두 “진짜 이미지"라고 레이블을 생성
    misleading_targets = np.zeros((batch_size, 1))

    # generator를 훈련(gan 모델에서 discriminator의 가중치는 동결)
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    start += batch_size
    if start > len(x_train) - batch_size:
      start = 0

    if step % 100 == 0:
        # 모델 가중치를 저장
        gan.save_weights('gan.h5')

        # 측정 지표를 출력
        print('스텝 %s에서 판별자 손실: %s' % (step, d_loss))
        print('스텝 %s에서 적대적 손실: %s' % (step, a_loss))

        # 생성된 이미지 하나를 저장
        img = image.array_to_img(generated_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'generated_car' + str(step) + '.png'))


스텝 0에서 판별자 손실: 0.6960480213165283
스텝 0에서 적대적 손실: 0.6979990005493164
스텝 100에서 판별자 손실: 0.7712632417678833
스텝 100에서 적대적 손실: 0.8585919141769409
스텝 200에서 판별자 손실: 0.6875166296958923
스텝 200에서 적대적 손실: 0.7941431403160095
스텝 300에서 판별자 손실: 0.8933683633804321
스텝 300에서 적대적 손실: 3.566394090652466
스텝 400에서 판별자 손실: 0.6932035684585571
스텝 400에서 적대적 손실: 0.739726185798645
스텝 500에서 판별자 손실: 0.6993890404701233
스텝 500에서 적대적 손실: 0.7418013215065002
스텝 600에서 판별자 손실: 0.7021721601486206
스텝 600에서 적대적 손실: 0.7573472261428833
스텝 700에서 판별자 손실: 0.6971556544303894
스텝 700에서 적대적 손실: 0.7259173393249512
스텝 800에서 판별자 손실: 0.7312043905258179
스텝 800에서 적대적 손실: 0.7983689904212952
스텝 900에서 판별자 손실: 0.7058581709861755
스텝 900에서 적대적 손실: 0.7521222233772278
스텝 1000에서 판별자 손실: 0.6856123805046082
스텝 1000에서 적대적 손실: 1.4617040157318115
스텝 1100에서 판별자 손실: 0.682522177696228
스텝 1100에서 적대적 손실: 0.7580477595329285
스텝 1200에서 판별자 손실: 0.693409264087677
스텝 1200에서 적대적 손실: 0.7686008214950562
스텝 1300에서 판별자 손실: 0.6895973086357117
스텝 1300에서 적대적 손실: 0.75089424